# OpenAI Parameters

# Overview
When making requests to OpenAI models, several parameters can be used to control the behavior and output of the model. \
Understanding these parameters helps in fine-tuning the responses to meet specific requirements, whether for generating text, answering questions, or any other use case.

For more detailed examples, refer to the official documentation [Azure OpenAI Service](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference)


In [12]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv(override=True)

azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
api_key = os.environ.get("AZURE_OPENAI_API_KEY")

client = AzureOpenAI(
  azure_endpoint=azure_endpoint, 
  api_key=api_key,
  api_version="2024-02-01"
)

CHAT_COMPLETIONS_MODEL = "gpt-4o-mini"
SEED = 123

# Parameter: max_tokens
**Description**: The maximum number of tokens to generate in the completion. \
**Default**: 16 \
**Example**: max_tokens=50

The token count of your prompt plus max_tokens can't exceed the model's context length. \
Most models have a context length of 2048 tokens (except for the newest models, which support 4096). Please refer to documentation.

In [13]:
def call_openai_with_max_tokens(max_tokens):
    response = client.chat.completions.create(
          model="gpt-4o-mini",
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "The best pet is a"}],
                    max_tokens=max_tokens
    )
    return response

# Generate with different presence_penalty values
penalties = [16, 32, 60, 100]
for penalty in penalties:
    print(f"Max Tokens: {penalty}\n")
    print(call_openai_with_max_tokens(penalty))
    print("\n" + "-"*80 + "\n")

Max Tokens: 16

ChatCompletion(id='chatcmpl-As71hbrNnOvYZYRzdocKXUEnvU1ZI', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='The best pet can vary greatly depending on individual preferences and lifestyles! Some people love', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1737460085, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_5154047bf2', usage=CompletionUsage(completion_tokens=16, prompt_tokens=22, total_tokens=38, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(aud

# Parameter: temperature

**Description**: Controls the randomness of the output. Lower values make the output more deterministic, while higher values increase randomness. \
**Value Range**: 0 to 1 \
**Default Value**: 1 \
**Example**: temperature=0.7

Higher values means the model will take more risks. \
Try 0.9 for more creative applications, and 0 (argmax sampling) for ones with a well-defined answer.

---
**NOTE**: We generally recommend altering this or top_p but not both.


In [14]:
def call_openai(num_times, prompt, temperature=0.75, use_seed=False):
    for i in range(num_times):
        if use_seed:
            response = client.chat.completions.create(
                model= CHAT_COMPLETIONS_MODEL,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                            {"role":"user","content": prompt}],
                    max_tokens=60,
                    seed=SEED,
                    temperature = temperature
            )
        else:
            response = client.chat.completions.create(
                model= CHAT_COMPLETIONS_MODEL,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                            {"role":"user","content": prompt}],
                    max_tokens=60,
                    temperature = temperature
            )
        print(response.choices[0].message.content)

In [15]:
# Without seed and temperature, the response is different each time
call_openai(10, 'The best pet is a ')

The best pet depends on individual preferences and lifestyles! Here are a few popular options:

1. **Dogs** - Known for their loyalty and companionship. They require regular exercise and social interaction.
2. **Cats** - Independent yet affectionate, cats are often lower maintenance than dogs and can be great for
The best pet is subjective and depends on individual preferences, lifestyles, and needs. Some people might say a dog is the best pet because of their loyalty and companionship, while others might prefer cats for their independence and low maintenance. Some may choose pets like rabbits, birds, or fish for their unique qualities
The best pet can vary widely depending on individual preferences, lifestyles, and living situations. Here are a few popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are often considered "man's best friend." They require regular exercise and attention.

2. **Cats**: Independent
The best pet can vary depending on individual pr

In [16]:
# Now using a seed and 0 temperature, the response is the much more consisitent
call_openai(10, 'The best pet is a ', temperature = 0, use_seed=True)

The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Here are a few popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are often considered "man's best friend." They require regular exercise and social interaction.

2. **Cats**:
The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Here are a few popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are often considered "man's best friend." They require regular exercise and social interaction.

2. **Cats**:
The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Here are a few popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are great for active individuals or families. They require regular exercise and social interaction.

2. **Cats**:
The best pet can vary greatly depending on individual preferences, lifestyles,

# Parameter: n
**Description**: Specifies the number of completions to generate for each prompt. \
**Default Value**: 1 \
**Example**: n = 3 

---
**Note**: Because this parameter generates many completions, it can quickly consume your token quota. Use carefully and ensure that you have reasonable settings for max_tokens and stop.

In [24]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages=[{"role": "system", "content": "You are a helpful assistant."},
              {"role": "user", "content": "The best pet is a "}],
    max_tokens=100,
    temperature=0.75,
    n=3
)

for index, c in enumerate(response.choices):
    print(index, c.message.content)

0 The best pet can vary depending on individual preferences, lifestyle, and needs. Here are a few options that people often consider as great pets:

1. **Dogs**: Known for their loyalty and companionship, dogs are often referred to as "man's best friend." They require regular exercise and social interaction.

2. **Cats**: Independent yet affectionate, cats can be great for people who may not have as much time for a pet. They are generally low-maintenance and can be very playful.

3
1 The best pet can vary depending on individual preferences, living situations, and lifestyles. Here are some popular options:

1. **Dog**: Known for their loyalty and companionship, dogs are often considered "man's best friend." They require regular exercise and social interaction.

2. **Cat**: Cats are independent and low-maintenance pets. They can be affectionate and playful while also being content to entertain themselves.

3. **Fish**: Fish are calming to watch and require less interaction than mammals.

# Parameter: presence_penalty
**Description**: Penalizes new tokens based on whether they appear in the text so far, encouraging the model to use new tokens. \
**Value Range**: -2.0 to 2.0 \
**Default Value**: 0 \
**Example**: presence_penalty=0.5

In [25]:
def call_openai_with_presence_penalty(presence_penalty):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "The best pet is a"}],
                    max_tokens=60,
                    presence_penalty= presence_penalty
    )
    return response

# Generate with different presence_penalty values
penalties = [0, 0.5, 1.0, 1.5, 2.0]
for penalty in penalties:
    print(f"Presence Penalty: {penalty}\n")
    print(call_openai_with_presence_penalty(penalty))
    print("\n" + "-"*80 + "\n")

Presence Penalty: 0

ChatCompletion(id='chatcmpl-As7Cz92LPb6GAnSL0yh6rZrWJzS9D', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='The best pet can vary greatly depending on individual preferences, lifestyle, and circumstances. Some popular choices include:\n\n1. **Dogs**: Known for their loyalty and companionship. They require daily exercise and social interaction.\n2. **Cats**: Independent yet affectionate, they are often easier to care for in', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1737460785, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_f3927aa00d', usage=CompletionUsage(completion_tokens=60, pr

# Parameter: frequency_penalty
**Description**: Penalizes new tokens based on their existing frequency in the text so far, reducing the likelihood of repeating the same line verbatim. \
**Value Range**: -2.0 to 2.0 \
**Default Value**: 0 \
**Example**: frequency_penalty=0.5

#### Use cases to explore
1. **Compare Responses** \
Generate multiple completions to compare and choose the best response for your use case.

2. **Increase Diversity** \
Use multiple completions to get a variety of responses, which is useful in creative applications.

3. **Enhance Robustness** \
Generate multiple responses to ensure consistency and accuracy across different completions.

#### Best Practices
1. **Optimize Prompt Length** \
Keep your prompts concise but informative to ensure the model has enough context.

2. **Adjust Temperature and Top_p** \
Use these parameters to balance between deterministic and creative responses.

3. **Monitor Token Usage** \
Be mindful of the max_tokens parameter to manage costs and response length.

4. **Use Stopping Sequences** \
Define stopping sequences to control where the model should stop generating text, ensuring the output is within the desired context.

5. **Generate Multiple Completions** \
Use the n parameter to generate multiple completions and select the best one for your needs.

## Exercise 1: Exploring 'top_p' Parameter
### Description:
- The 'top_p' parameter controls nucleus sampling. Lower values make the output more focused by limiting the pool of tokens.

### Task:
- Write a function that calls the model with different 'top_p' values and observe how the output changes.


In [26]:
def call_openai_with_top_p(top_p):
    response = client.chat.completions.create(
          model= CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "Describe the benefits of exercise."}],
                    max_tokens=60,
                    top_p= top_p
    )
    return response

# Try with different values of top_p
top_p_values = [0.1, 0.3, 0.7, 1.0]
for value in top_p_values:
    print(f"Top P: {value}\n")
    print(call_openai_with_top_p(value))
    print("\n" + "-"*80 + "\n")


Top P: 0.1

ChatCompletion(id='chatcmpl-As7DdkBmtU5ykafi9uFq2PreaHm2S', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='Exercise offers a wide range of benefits for both physical and mental health. Here are some of the key advantages:\n\n### Physical Health Benefits:\n1. **Improved Cardiovascular Health**: Regular exercise strengthens the heart, improves circulation, and can lower blood pressure and cholesterol levels, reducing the risk of heart', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1737460825, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_f3927aa00d', usage=CompletionUsage(completion_tokens=60, 

## Exercise 2: Frequency Penalty Exploration
### Description:
- This exercise will explore how the 'frequency_penalty' parameter affects the model's tendency to repeat itself.

### Task:
- Generate completions using different frequency_penalty values and note the diversity in responses.



In [27]:
def call_openai_with_frequency_penalty(frequency_penalty):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "Tell me a fun fact about space."}],
                    max_tokens=60,
                    frequency_penalty= frequency_penalty
    )
    return response

penalties = [-1.0, 0.0, 1.0, 2.0]
for penalty in penalties:
    print(f"Frequency Penalty: {penalty}\n")
    print(call_openai_with_frequency_penalty(penalty))
    print("\n" + "-"*80 + "\n")


Frequency Penalty: -1.0

ChatCompletion(id='chatcmpl-As7DyI7GW7a05IAMZnoRWOBSuBmOQ', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='One fun fact about space is that a day on Venus is longer than a year on Venus! Venus has a very slow rotation on its axis, taking about 243 Earth days to complete one full rotation. In contrast, it only takes about 225 Earth days to orbit the Sun. This means that', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1737460846, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_5154047bf2', usage=CompletionUsage(completion_tokens=60, prompt_tokens=25, total_tokens=85, completion_tokens

## Exercise 3: Multi-Completion with 'n' Parameter
### Description:
- This exercise will help students generate multiple completions for the same prompt and compare the results.

### Task:
- Generate 3 completions for the same prompt by adjusting the 'n' parameter.


In [35]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages=[{"role": "system", "content": "You are a helpful assistant."},
              {"role": "user", "content": "Tell me a fun fact about dinosaurs."}],
    max_tokens=100,
    temperature=0.9,
    n=3
)

for index, c in enumerate(response.choices):
    print(index, c.message.content)

0 Sure! One fun fact about dinosaurs is that the largest dinosaurs, like the Argentinosaurus, could weigh as much as 100 tons, which is roughly equivalent to about 14 adult elephants! These massive herbivores roamed the Earth during the Late Cretaceous period, around 100 million years ago. Despite their size, they were primarily plant-eaters and had long necks that allowed them to reach high vegetation.
1 One fun fact about dinosaurs is that not all of them were gigantic! While many popular dinosaurs, like Tyrannosaurus rex and Brachiosaurus, are known for their massive sizes, some dinosaurs were quite small. For example, the Compsognathus was about the size of a chicken, measuring around 3 feet (1 meter) long. This tiny dinosaur was a carnivore and primarily fed on small insects and lizards. The diversity in size among dinosaurs showcases just how varied these
2 Sure! Here's a fun fact about dinosaurs: The largest dinosaur known to exist is believed to be the Argentinosaurus, which co

## Exercise 4: Temperature vs. Deterministic Output
### Description:
- Adjust the 'temperature' parameter to explore the balance between creative and deterministic responses.

### Task:
- Generate completions with different temperature values and analyze the variance in output.


In [36]:
def call_openai_with_temperature(temperature):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "Suggest a name for a new tech startup."}],
                    max_tokens=30,
                    temperature= temperature
    )
    return response

temperatures = [0.0, 0.3, 0.7, 1.0]
for temp in temperatures:
    print(f"Temperature: {temp}\n")
    print(call_openai_with_temperature(temp))
    print("\n" + "-"*80 + "\n")

Temperature: 0.0

ChatCompletion(id='chatcmpl-As7GlMwgPBpiOALCvYtInytLQbe8S', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='Sure! Here are some suggestions for a tech startup name:\n\n1. **TechNova**\n2. **InnoSphere**\n3. **ByteBridge', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1737461019, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_5154047bf2', usage=CompletionUsage(completion_tokens=30, prompt_tokens=26, total_tokens=56, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTo